In [40]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Generate realistic OHLCV data for a fictitious stock for 100 days
n_days = 100
dates = pd.date_range('2023-01-01', periods=n_days, freq='D')

# Generate prices with random walk
price = np.cumsum(np.random.randn(n_days)) + 100  # Start around 100
high = price + np.random.rand(n_days) * 2  # High is slightly above price
low = price - np.random.rand(n_days) * 2  # Low is slightly below price
open_ = price + np.random.randn(n_days) * 0.5  # Open around price with noise
close = price + np.random.randn(n_days) * 0.5  # Close around price with noise

# Generate random volume
volume = np.random.randint(1000, 10000, size=n_days)

# Create a DataFrame with the OHLCV data
ohlcv_df = pd.DataFrame({
    'Date': dates,
    'Open': open_,
    'High': high,
    'Low': low,
    'Close': close,
    'Volume': volume
})

ohlcv_df.set_index('Date', inplace=True)
ohlcv_df.head()  # Show the first few rows


,Open,High,Low,Close,Volume
Date,,,,,
2023-01-01,101.130170,101.331536,98.907092,100.487458,4900
2023-01-02,100.004615,100.802665,99.353176,100.214121,7677
2023-01-03,101.228048,101.245869,99.852331,101.167498,7981
2023-01-04,102.916485,103.204399,101.544133,102.115553,7376
2023-01-05,101.831550,104.180834,101.904529,102.554688,1523


In [41]:
from eth_rpc import PrivateKeyWallet
from emp.orderly.utils import from_address
from emp.orderly import EmpOrderly, EmpyrealOrderlySDK

wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

In [52]:
import pandas as pd
import pandas_ta as ta
from emp.orderly import Strategy
from typing import Union


class MTCVB(Strategy):
    order_size: float = 0.5

    def MACD(close: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> pd.DataFrame:
        """
        Returns `n`-period Moving Average Convergence Divergence of array `close`.
        """
        return ta.macd(close, fast, slow, signal)

    def ATR(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14) -> pd.Series:
        """
        Returns `n`-period Average True Range of array `high, low, close`.
        """
        return ta.atr(high, low, close, length)
    
    def RSI(close: pd.Series, length: int = 14) -> Union[pd.DataFrame, pd.Series]:
        """
        Returns `n`-period Relative Strength Index of array `high, low, close`.
        """
        return ta.rsi(close, length)
    
    def init(self):
        self.macd = self.MACD(self.data.close)
        self.atr = self.ATR(self.data.high, self.data.low, self.data.close)
        self.rsi = self.RSI(self.data.close)
        self.volatility_threshold = self.atr.rolling(window=14).mean() * 1.5

    def next(self):
        if (
            (self.macd["MACD_12_26_9"] > self.macd["MACDs_12_26_9"]) &
            (self.rsi > 30) &
            (self.rsi < 70) &
            (self.atr > self.volatility_threshold)
        ):
            self.position.close()
            self.buy(size = self.order_size)
        elif (
            (self.macd["MACD_12_26_9"] < self.macd["MACDs_12_26_9"]) &
            (self.rsi > 30) &
            (self.rsi < 70) &
            (self.atr > self.volatility_threshold)
        ):
            self.position.close()
            self.buy(size=self.order_size)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

from emp.orderly_types import PerpetualAssetType, Interval
from emp.orderly.onboarding.faucet import request_testnet_funds

# get some testnet USDC from the faucet
await request_testnet_funds(wallet)
print(await wallet.balance())

# load strategy and data
tester.set_strategy(MTCVB)
await tester.load_data(
    lookback=5,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.BTC,
)
tester.backtest()

0
                        open     high      low    close  volume_asset  volume
timestamp                                                                    
2024-09-24 19:20:00  62865.5  62865.5  62865.5  62865.5           0.0     0.0
2024-09-24 19:25:00  62865.5  62865.5  62865.5  62865.5           0.0     0.0
2024-09-24 19:30:00  62865.5  62865.5  62865.5  62865.5           0.0     0.0
2024-09-24 19:35:00  62865.5  62865.5  62865.5  62865.5           0.0     0.0
2024-09-24 19:40:00  62865.5  62865.5  62865.5  62865.5           0.0     0.0


/workspaces/orderly-test-ground/emp/orderly/backtest/backtesting.py:1175: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  self._init_data(self._history)


TypeError: only length-1 arrays can be converted to Python scalars